# Лабораторная работа №1

### 1. Численные методы решения СЛАУ

In [4]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Методы численного решения СЛАУ


Добавим СЛАУ с которой будет работать

In [6]:
# Из задачника Аристова-Лобанова пункт Д)
A = np.zeros((100, 100))
b = np.zeros(100)
n = len(b)

for k in range(n):
    start_j = max(0, k - 4)
    end_j = min(99, k + 4)
    
    for j in range(start_j, end_j + 1):
        if j == k:
            A[k, j] = 10
        else:
            A[k, j] = 1
    
    b[k] = k + 1

Выведем матрицу, чтобы убедиться в её корректности

In [9]:
for i in range(0, len(A)):
    print(A[i])

[10.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 1. 10.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 1.  1. 10.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.